# Minimum EigenValue For 4x4 Matrix

In [141]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute, Aer
import pennylane as qml
import numpy as np
from numpy import pi
simulator = Aer.get_backend('qasm_simulator')
from scipy.linalg import expm, sinm, cosm



# Matrix and Decomposition

In [1336]:
M= np.array(
[[3+4.7,  3.,  6.,  2.],
       [ 3.,  1+4.7,  -4.,  3.],
       [ 6.,  -4.,  5+4.7,  -2.],
       [ 2.,3.,  -2.,  8+4.7]])

In [1337]:
M

array([[ 7.7,  3. ,  6. ,  2. ],
       [ 3. ,  5.7, -4. ,  3. ],
       [ 6. , -4. ,  9.7, -2. ],
       [ 2. ,  3. , -2. , 12.7]])

In [1338]:
#M= np.matrix('1 0 0 0 ;0 0 -1 0;0 -1 0 0;0 0 0 1')

In [1339]:
coeffs, obs_list = qml.utils.decompose_hamiltonian(M)
H = qml.Hamiltonian(coeffs, obs_list)

In [1340]:
print(H)

(8.95) [I0 I1]
+ (0.5) [I0 X1]
+ (-0.25) [I0 Z1]
+ (4.5) [X0 I1]
+ (-1.0) [X0 X1]
+ (1.5) [X0 Z1]
+ (-3.0) [Y0 Y1]
+ (-2.2499999999999996) [Z0 I1]
+ (2.5) [Z0 X1]
+ (1.25) [Z0 Z1]


In [1341]:
coef(H)

[8.95,
 1.25,
 -1.0,
 -3.0,
 0.5,
 4.5,
 0,
 0,
 -0.25,
 -2.2499999999999996,
 0,
 0,
 1.5,
 2.5,
 0,
 0]

In [1342]:
def coef(H):
    coeff = [0]*16
    h = str(H)
    for i in range (len(h)):
        p = ""
        if h[i] =="(":
            j = i+1 
            while h[j]!=")" :
                p=p+h[j]
                j+=1
            
            if h[j+3] == "I" and h[j+6] == "I":
                coeff[0] = float(p)
                
            if h[j+3] == "Z" and h[j+6] == "Z":
                coeff[1] = float(p)
                
            if h[j+3] == "X" and h[j+6] == "X":
                coeff[2] = float(p)
                
            if h[j+3] == "Y" and h[j+6] == "Y":
                coeff[3] = float(p)
            
            if h[j+3] == "I" and h[j+6] == "X":
                coeff[4] = float(p)
                
            if h[j+3] == "X" and h[j+6] == "I":
                coeff[5] = float(p)
                
            if h[j+3] == "I" and h[j+6] == "Y":
                coeff[6] = float(p)
                
            if h[j+3] == "Y" and h[j+6] == "I":
                coeff[7] = float(p)
                
            if h[j+3] == "I" and h[j+6] == "Z":
                coeff[8] = float(p)
                
            if h[j+3] == "Z" and h[j+6] == "I":
                coeff[9] = float(p)
                
            if h[j+3] == "X" and h[j+6] == "Y":
                coeff[10] = float(p)
                
            if h[j+3] == "X" and h[j+6] == "Y":
                coeff[11] = float(p)
                
            if h[j+3] == "X" and h[j+6] == "Z":
                coeff[12] = float(p)
                
            if h[j+3] == "Z" and h[j+6] == "X":
                coeff[13] = float(p)
                
            if h[j+3] == "Y" and h[j+6] == "Z":
                coeff[14] = float(p)
                
            if h[j+3] == "Z" and h[j+6] == "Y":
                coeff[15] = float(p)
                
    return(coeff)
            
            

# Ansatz

In [1343]:
def Operational_circuit(params):
    qc = QuantumCircuit(2)
    
    #qc.h(range(2))
    qc.rx(params[0],0)
    qc.rx(params[1],1)
    qc.ry(params[2],0)
    qc.ry(params[3],1)
    qc.crx(params[8],0,1)
    qc.ry(params[4],0)
    qc.ry(params[5],1)
    qc.rx(params[6],0)
    qc.rx(params[7],1)
    
    return qc
    

In [1344]:
Operational_circuit_try = Operational_circuit([1,2,3,4,5,6,7,8,9])

In [1345]:
Operational_circuit_try.draw()

┌───────┐┌───────┐         ┌───────┐┌───────┐
q_0: ┤ RX(1) ├┤ RY(3) ├────■────┤ RY(5) ├┤ RX(7) ├
     ├───────┤├───────┤┌───┴───┐├───────┤├───────┤
q_1: ┤ RX(2) ├┤ RY(4) ├┤ RX(9) ├┤ RY(6) ├┤ RX(8) ├
     └───────┘└───────┘└───────┘└───────┘└───────┘

# Measurments for different components of H

# 1. ZZ

In [1346]:
def measure_zz_circuit(given_circuit):
    zz_meas = given_circuit.copy()
    zz_meas.measure_all()
    return zz_meas

zz_meas = measure_zz_circuit(Operational_circuit_try)
zz_meas.draw()

┌───────┐┌───────┐         ┌───────┐┌───────┐ ░ ┌─┐   
   q_0: ┤ RX(1) ├┤ RY(3) ├────■────┤ RY(5) ├┤ RX(7) ├─░─┤M├───
        ├───────┤├───────┤┌───┴───┐├───────┤├───────┤ ░ └╥┘┌─┐
   q_1: ┤ RX(2) ├┤ RY(4) ├┤ RX(9) ├┤ RY(6) ├┤ RX(8) ├─░──╫─┤M├
        └───────┘└───────┘└───────┘└───────┘└───────┘ ░  ║ └╥┘
meas_0: ═════════════════════════════════════════════════╩══╬═
                                                            ║ 
meas_1: ════════════════════════════════════════════════════╩═

In [1347]:
def measure_zz(given_circuit, num_shots = 10000):

    zz_meas = measure_zz_circuit(given_circuit)
    
    result = execute(zz_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(zz_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0 

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    zz = counts['00'] + counts['11'] - counts['01'] - counts['10']
    zz = zz / total_counts
    
    return zz

In [1348]:
zz = measure_zz(Operational_circuit_try)
print("<ZZ> =", str(zz))

<ZZ> = -0.365


# 2. XX

In [1349]:
def measure_xx_circuit(given_circuit):
    xx_meas = given_circuit.copy()
    xx_meas.h(0)
    xx_meas.h(1)
    xx_meas.measure_all()

    return xx_meas

In [1350]:
xx_meas = measure_xx_circuit(Operational_circuit_try)
xx_meas.draw()

┌───────┐┌───────┐         ┌───────┐┌───────┐┌───┐ ░ ┌─┐   
   q_0: ┤ RX(1) ├┤ RY(3) ├────■────┤ RY(5) ├┤ RX(7) ├┤ H ├─░─┤M├───
        ├───────┤├───────┤┌───┴───┐├───────┤├───────┤├───┤ ░ └╥┘┌─┐
   q_1: ┤ RX(2) ├┤ RY(4) ├┤ RX(9) ├┤ RY(6) ├┤ RX(8) ├┤ H ├─░──╫─┤M├
        └───────┘└───────┘└───────┘└───────┘└───────┘└───┘ ░  ║ └╥┘
meas_0: ══════════════════════════════════════════════════════╩══╬═
                                                                 ║ 
meas_1: ═════════════════════════════════════════════════════════╩═

In [1351]:
def measure_xx(given_circuit, num_shots = 10000):
    
    xx_meas = measure_xx_circuit(given_circuit)
    
    result = execute(xx_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(xx_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    xx = counts['00'] + counts['11'] - counts['01'] - counts['10']
    xx = xx / total_counts
    
    return xx

In [1352]:
xx = measure_xx(Operational_circuit_try)
print("<XX> =", str(xx))

<XX> = 0.5234


# 3. YY

In [1353]:
def measure_yy_circuit(given_circuit):
    yy_meas = given_circuit.copy()
    yy_meas.rz(-pi/2,0)
    yy_meas.rz(-pi/2,1)

    yy_meas.h(0)
    yy_meas.h(1)
    yy_meas.measure_all()

    return yy_meas

In [1354]:
yy_meas = measure_yy_circuit(Operational_circuit_try)
yy_meas.draw()

┌───────┐┌───────┐         ┌───────┐┌───────┐┌───────────┐┌───┐ ░ ┌─┐»
   q_0: ┤ RX(1) ├┤ RY(3) ├────■────┤ RY(5) ├┤ RX(7) ├┤ RZ(-pi/2) ├┤ H ├─░─┤M├»
        ├───────┤├───────┤┌───┴───┐├───────┤├───────┤├───────────┤├───┤ ░ └╥┘»
   q_1: ┤ RX(2) ├┤ RY(4) ├┤ RX(9) ├┤ RY(6) ├┤ RX(8) ├┤ RZ(-pi/2) ├┤ H ├─░──╫─»
        └───────┘└───────┘└───────┘└───────┘└───────┘└───────────┘└───┘ ░  ║ »
meas_0: ═══════════════════════════════════════════════════════════════════╩═»
                                                                             »
meas_1: ═════════════════════════════════════════════════════════════════════»
                                                                             »
«           
«   q_0: ───
«        ┌─┐
«   q_1: ┤M├
«        └╥┘
«meas_0: ═╬═
«         ║ 
«meas_1: ═╩═
«

In [1355]:
def measure_yy(given_circuit, num_shots = 10000):
    
    yy_meas = measure_yy_circuit(given_circuit)
    
    result = execute(yy_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(yy_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    yy = counts['00'] + counts['11'] - counts['01'] - counts['10']
    yy = yy / total_counts
    
    return yy

In [1356]:
yy = measure_yy(Operational_circuit_try)
print("<yy> =", str(yy))

<yy> = 0.769


# 4. IX

In [1357]:
def measure_ix_circuit(given_circuit):
    ix_meas = given_circuit.copy()
    ix_meas.h(1)
    ix_meas.measure_all()

    return ix_meas
def measure_ix(given_circuit, num_shots = 10000):
    
    ix_meas = measure_ix_circuit(given_circuit)
    
    result = execute(ix_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(ix_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    ix = counts['00'] - counts['11'] + counts['01'] - counts['10']
    ix = ix / total_counts
    
    return ix

In [1358]:
ix_meas = measure_ix_circuit(Operational_circuit_try)
ix_meas.draw()

┌───────┐┌───────┐         ┌───────┐┌───────┐      ░ ┌─┐   
   q_0: ┤ RX(1) ├┤ RY(3) ├────■────┤ RY(5) ├┤ RX(7) ├──────░─┤M├───
        ├───────┤├───────┤┌───┴───┐├───────┤├───────┤┌───┐ ░ └╥┘┌─┐
   q_1: ┤ RX(2) ├┤ RY(4) ├┤ RX(9) ├┤ RY(6) ├┤ RX(8) ├┤ H ├─░──╫─┤M├
        └───────┘└───────┘└───────┘└───────┘└───────┘└───┘ ░  ║ └╥┘
meas_0: ══════════════════════════════════════════════════════╩══╬═
                                                                 ║ 
meas_1: ═════════════════════════════════════════════════════════╩═

In [1359]:
ix = measure_ix(Operational_circuit_try)
print("<IX> =", str(ix))

<IX> = 0.4182


# 5. XI

In [1360]:
def measure_xi_circuit(given_circuit):
    xi_meas = given_circuit.copy()
    xi_meas.h(0)
    xi_meas.measure_all()

    return xi_meas
def measure_xi(given_circuit, num_shots = 10000):
    xi_meas = measure_xi_circuit(given_circuit)
    
    result = execute(xi_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(xi_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    xi = counts['00'] - counts['11'] - counts['01'] + counts['10']
    xi = xi / total_counts
    
    return xi
xi = measure_xi(Operational_circuit_try)
print("<XI> =", str(xi))

<XI> = 0.5816


In [1361]:
A = measure_xi_circuit(Operational_circuit_try)
A.draw()

┌───────┐┌───────┐         ┌───────┐┌───────┐┌───┐ ░ ┌─┐   
   q_0: ┤ RX(1) ├┤ RY(3) ├────■────┤ RY(5) ├┤ RX(7) ├┤ H ├─░─┤M├───
        ├───────┤├───────┤┌───┴───┐├───────┤├───────┤└───┘ ░ └╥┘┌─┐
   q_1: ┤ RX(2) ├┤ RY(4) ├┤ RX(9) ├┤ RY(6) ├┤ RX(8) ├──────░──╫─┤M├
        └───────┘└───────┘└───────┘└───────┘└───────┘      ░  ║ └╥┘
meas_0: ══════════════════════════════════════════════════════╩══╬═
                                                                 ║ 
meas_1: ═════════════════════════════════════════════════════════╩═

# 6. IY

In [1362]:
def measure_iy_circuit(given_circuit):
    iy_meas = given_circuit.copy()
    iy_meas.rz(-pi/2,1)
    iy_meas.h(1)
    iy_meas.measure_all()

    return iy_meas
def measure_iy(given_circuit, num_shots = 10000):
    
    iy_meas = measure_iy_circuit(given_circuit)
    
    result = execute(iy_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(iy_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    iy = counts['00'] - counts['11'] + counts['01'] - counts['10']
    iy = iy / total_counts
    
    return iy
iy = measure_iy(Operational_circuit_try)
print("<IY> =", str(iy))

<IY> = 0.253


In [1363]:
A = measure_iy_circuit(Operational_circuit_try)
A.draw()

┌───────┐┌───────┐         ┌───────┐┌───────┐                   ░ ┌─┐»
   q_0: ┤ RX(1) ├┤ RY(3) ├────■────┤ RY(5) ├┤ RX(7) ├───────────────────░─┤M├»
        ├───────┤├───────┤┌───┴───┐├───────┤├───────┤┌───────────┐┌───┐ ░ └╥┘»
   q_1: ┤ RX(2) ├┤ RY(4) ├┤ RX(9) ├┤ RY(6) ├┤ RX(8) ├┤ RZ(-pi/2) ├┤ H ├─░──╫─»
        └───────┘└───────┘└───────┘└───────┘└───────┘└───────────┘└───┘ ░  ║ »
meas_0: ═══════════════════════════════════════════════════════════════════╩═»
                                                                             »
meas_1: ═════════════════════════════════════════════════════════════════════»
                                                                             »
«           
«   q_0: ───
«        ┌─┐
«   q_1: ┤M├
«        └╥┘
«meas_0: ═╬═
«         ║ 
«meas_1: ═╩═
«

# 7. YI

In [1364]:
def measure_yi_circuit(given_circuit):
    yi_meas = given_circuit.copy()
    yi_meas.rz(-pi/2,0)
    yi_meas.h(0)
    yi_meas.measure_all()

    return yi_meas
def measure_yi(given_circuit, num_shots = 10000):
    
    yi_meas = measure_yi_circuit(given_circuit)
    
    result = execute(yi_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(yi_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    yi = counts['00'] - counts['11'] - counts['01'] + counts['10']
    yi = yi / total_counts
    
    return yi

yi = measure_yi(Operational_circuit_try)
print("<YI> =", str(yi))

<YI> = 0.0968


In [1365]:
A = measure_yi_circuit(Operational_circuit_try)
A.draw()

┌───────┐┌───────┐         ┌───────┐┌───────┐┌───────────┐┌───┐ ░ ┌─┐»
   q_0: ┤ RX(1) ├┤ RY(3) ├────■────┤ RY(5) ├┤ RX(7) ├┤ RZ(-pi/2) ├┤ H ├─░─┤M├»
        ├───────┤├───────┤┌───┴───┐├───────┤├───────┤└───────────┘└───┘ ░ └╥┘»
   q_1: ┤ RX(2) ├┤ RY(4) ├┤ RX(9) ├┤ RY(6) ├┤ RX(8) ├───────────────────░──╫─»
        └───────┘└───────┘└───────┘└───────┘└───────┘                   ░  ║ »
meas_0: ═══════════════════════════════════════════════════════════════════╩═»
                                                                             »
meas_1: ═════════════════════════════════════════════════════════════════════»
                                                                             »
«           
«   q_0: ───
«        ┌─┐
«   q_1: ┤M├
«        └╥┘
«meas_0: ═╬═
«         ║ 
«meas_1: ═╩═
«

# 8. IZ

In [1366]:
def measure_iz(given_circuit, num_shots = 10000):

    zz_meas = measure_zz_circuit(given_circuit)
    
    result = execute(zz_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(zz_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0 

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    zz = counts['00'] - counts['11']+ counts['01'] - counts['10']
    zz = zz / total_counts
    
    return zz

iz = measure_iz(Operational_circuit_try)
print("<IZ> =", str(iz))

<IZ> = 0.394


# 9. ZI

In [1367]:
def measure_zi(given_circuit, num_shots = 10000):

    zz_meas = measure_zz_circuit(given_circuit)
    
    result = execute(zz_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(zz_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0 

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    zz = counts['00'] - counts['11']- counts['01'] + counts['10']
    zz = zz / total_counts
    
    return zz

zi = measure_zi(Operational_circuit_try)
print("<ZI> =", str(zi))

<ZI> = 0.1822


# 10. XY

In [1368]:
def measure_xy_circuit(given_circuit):
    xy_meas = given_circuit.copy()
    xy_meas.rz(-pi/2,1)
    xy_meas.h(1)
    xy_meas.h(0)
    xy_meas.measure_all()

    return xy_meas

def measure_xy(given_circuit, num_shots = 10000):

    zz_meas = measure_xy_circuit(given_circuit)
    
    result = execute(zz_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(zz_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0 

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    zz = counts['00'] + counts['11'] - counts['01'] - counts['10']
    zz = zz / total_counts
    
    return zz

xy = measure_xy(Operational_circuit_try)
print("<XY> =", str(xy))

<XY> = 0.2604


In [1369]:
A = measure_xy_circuit(Operational_circuit_try)
A.draw()

┌───────┐┌───────┐         ┌───────┐┌───────┐    ┌───┐          ░ ┌─┐»
   q_0: ┤ RX(1) ├┤ RY(3) ├────■────┤ RY(5) ├┤ RX(7) ├────┤ H ├──────────░─┤M├»
        ├───────┤├───────┤┌───┴───┐├───────┤├───────┤┌───┴───┴───┐┌───┐ ░ └╥┘»
   q_1: ┤ RX(2) ├┤ RY(4) ├┤ RX(9) ├┤ RY(6) ├┤ RX(8) ├┤ RZ(-pi/2) ├┤ H ├─░──╫─»
        └───────┘└───────┘└───────┘└───────┘└───────┘└───────────┘└───┘ ░  ║ »
meas_0: ═══════════════════════════════════════════════════════════════════╩═»
                                                                             »
meas_1: ═════════════════════════════════════════════════════════════════════»
                                                                             »
«           
«   q_0: ───
«        ┌─┐
«   q_1: ┤M├
«        └╥┘
«meas_0: ═╬═
«         ║ 
«meas_1: ═╩═
«

# 11. YX

In [1370]:
def measure_yx_circuit(given_circuit):
    xy_meas = given_circuit.copy()
    xy_meas.rz(-pi/2,0)
    xy_meas.h(0)
    xy_meas.h(1)
    xy_meas.measure_all()

    return xy_meas

def measure_yx(given_circuit, num_shots = 10000):

    zz_meas = measure_yx_circuit(given_circuit)
    
    result = execute(zz_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(zz_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0 

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    zz = counts['00'] + counts['11'] - counts['01'] - counts['10']
    zz = zz / total_counts
    
    return zz

yx = measure_yx(Operational_circuit_try)
print("<YX> =", str(yx))

<YX> = -0.1896


In [1371]:
A = measure_yx_circuit(Operational_circuit_try)
A.draw()

┌───────┐┌───────┐         ┌───────┐┌───────┐┌───────────┐┌───┐ ░ ┌─┐»
   q_0: ┤ RX(1) ├┤ RY(3) ├────■────┤ RY(5) ├┤ RX(7) ├┤ RZ(-pi/2) ├┤ H ├─░─┤M├»
        ├───────┤├───────┤┌───┴───┐├───────┤├───────┤└───┬───┬───┘└───┘ ░ └╥┘»
   q_1: ┤ RX(2) ├┤ RY(4) ├┤ RX(9) ├┤ RY(6) ├┤ RX(8) ├────┤ H ├──────────░──╫─»
        └───────┘└───────┘└───────┘└───────┘└───────┘    └───┘          ░  ║ »
meas_0: ═══════════════════════════════════════════════════════════════════╩═»
                                                                             »
meas_1: ═════════════════════════════════════════════════════════════════════»
                                                                             »
«           
«   q_0: ───
«        ┌─┐
«   q_1: ┤M├
«        └╥┘
«meas_0: ═╬═
«         ║ 
«meas_1: ═╩═
«

# 12. XZ

In [1372]:
def measure_xz_circuit(given_circuit):
    xy_meas = given_circuit.copy()
    xy_meas.h(0)
    xy_meas.measure_all()

    return xy_meas

def measure_xz(given_circuit, num_shots = 10000):

    zz_meas = measure_xz_circuit(given_circuit)
    
    result = execute(zz_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(zz_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0 

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    zz = counts['00'] + counts['11'] - counts['01'] - counts['10']
    zz = zz / total_counts
    
    return zz

xz = measure_xz(Operational_circuit_try)
print("<XZ> =", str(xz))

<XZ> = 0.778


In [1373]:
A = measure_xz_circuit(Operational_circuit_try)
A.draw()

┌───────┐┌───────┐         ┌───────┐┌───────┐┌───┐ ░ ┌─┐   
   q_0: ┤ RX(1) ├┤ RY(3) ├────■────┤ RY(5) ├┤ RX(7) ├┤ H ├─░─┤M├───
        ├───────┤├───────┤┌───┴───┐├───────┤├───────┤└───┘ ░ └╥┘┌─┐
   q_1: ┤ RX(2) ├┤ RY(4) ├┤ RX(9) ├┤ RY(6) ├┤ RX(8) ├──────░──╫─┤M├
        └───────┘└───────┘└───────┘└───────┘└───────┘      ░  ║ └╥┘
meas_0: ══════════════════════════════════════════════════════╩══╬═
                                                                 ║ 
meas_1: ═════════════════════════════════════════════════════════╩═

# 13. ZX

In [1374]:
def measure_zx_circuit(given_circuit):
    xy_meas = given_circuit.copy()
    xy_meas.h(1)
    xy_meas.measure_all()

    return xy_meas

def measure_zx(given_circuit, num_shots = 10000):

    zz_meas = measure_zx_circuit(given_circuit)
    
    result = execute(zz_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(zz_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0 

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    zz = counts['00'] + counts['11'] - counts['01'] - counts['10']
    zz = zz / total_counts
    
    return zz

zx = measure_zx(Operational_circuit_try)
print("<ZX> =", str(zx))

<ZX> = 0.6958


In [1375]:
A = measure_zx_circuit(Operational_circuit_try)
A.draw()

┌───────┐┌───────┐         ┌───────┐┌───────┐      ░ ┌─┐   
   q_0: ┤ RX(1) ├┤ RY(3) ├────■────┤ RY(5) ├┤ RX(7) ├──────░─┤M├───
        ├───────┤├───────┤┌───┴───┐├───────┤├───────┤┌───┐ ░ └╥┘┌─┐
   q_1: ┤ RX(2) ├┤ RY(4) ├┤ RX(9) ├┤ RY(6) ├┤ RX(8) ├┤ H ├─░──╫─┤M├
        └───────┘└───────┘└───────┘└───────┘└───────┘└───┘ ░  ║ └╥┘
meas_0: ══════════════════════════════════════════════════════╩══╬═
                                                                 ║ 
meas_1: ═════════════════════════════════════════════════════════╩═

# 14. YZ

In [1376]:
def measure_yz_circuit(given_circuit):
    xy_meas = given_circuit.copy()
    xy_meas.rz(-pi/2,0)
    xy_meas.h(0)
    xy_meas.measure_all()

    return xy_meas

def measure_yz(given_circuit, num_shots = 10000):

    zz_meas = measure_yz_circuit(given_circuit)
    
    result = execute(zz_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(zz_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0 

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    zz = counts['00'] + counts['11'] - counts['01'] - counts['10']
    zz = zz / total_counts
    
    return zz

yz = measure_yz(Operational_circuit_try)
print("<YZ> =", str(yz))

<YZ> = -0.0566


In [1377]:
A = measure_yz_circuit(Operational_circuit_try)
A.draw()

┌───────┐┌───────┐         ┌───────┐┌───────┐┌───────────┐┌───┐ ░ ┌─┐»
   q_0: ┤ RX(1) ├┤ RY(3) ├────■────┤ RY(5) ├┤ RX(7) ├┤ RZ(-pi/2) ├┤ H ├─░─┤M├»
        ├───────┤├───────┤┌───┴───┐├───────┤├───────┤└───────────┘└───┘ ░ └╥┘»
   q_1: ┤ RX(2) ├┤ RY(4) ├┤ RX(9) ├┤ RY(6) ├┤ RX(8) ├───────────────────░──╫─»
        └───────┘└───────┘└───────┘└───────┘└───────┘                   ░  ║ »
meas_0: ═══════════════════════════════════════════════════════════════════╩═»
                                                                             »
meas_1: ═════════════════════════════════════════════════════════════════════»
                                                                             »
«           
«   q_0: ───
«        ┌─┐
«   q_1: ┤M├
«        └╥┘
«meas_0: ═╬═
«         ║ 
«meas_1: ═╩═
«

# 15. ZY

In [1378]:
def measure_zy_circuit(given_circuit):
    xy_meas = given_circuit.copy()
    xy_meas.rz(-pi/2,1)
    xy_meas.h(1)
    xy_meas.measure_all()

    return xy_meas

def measure_zy(given_circuit, num_shots = 10000):

    zz_meas = measure_zy_circuit(given_circuit)
    
    result = execute(zz_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(zz_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0 

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    zz = counts['00'] + counts['11'] - counts['01'] - counts['10']
    zz = zz / total_counts
    
    return zz

zy = measure_zy(Operational_circuit_try)
print("<ZY> =", str(zy))

<ZY> = 0.1616


In [1379]:
A = measure_zy_circuit(Operational_circuit_try)
A.draw()

┌───────┐┌───────┐         ┌───────┐┌───────┐                   ░ ┌─┐»
   q_0: ┤ RX(1) ├┤ RY(3) ├────■────┤ RY(5) ├┤ RX(7) ├───────────────────░─┤M├»
        ├───────┤├───────┤┌───┴───┐├───────┤├───────┤┌───────────┐┌───┐ ░ └╥┘»
   q_1: ┤ RX(2) ├┤ RY(4) ├┤ RX(9) ├┤ RY(6) ├┤ RX(8) ├┤ RZ(-pi/2) ├┤ H ├─░──╫─»
        └───────┘└───────┘└───────┘└───────┘└───────┘└───────────┘└───┘ ░  ║ »
meas_0: ═══════════════════════════════════════════════════════════════════╩═»
                                                                             »
meas_1: ═════════════════════════════════════════════════════════════════════»
                                                                             »
«           
«   q_0: ───
«        ┌─┐
«   q_1: ┤M├
«        └╥┘
«meas_0: ═╬═
«         ║ 
«meas_1: ═╩═
«

# Cost_Function

In [1380]:
def eigen_value(params):
    num_shots = 100000
    zz = measure_zz(Operational_circuit(params), num_shots = num_shots)
    xx = measure_xx(Operational_circuit(params), num_shots = num_shots)
    yy = measure_yy(Operational_circuit(params), num_shots = num_shots)
    ix = measure_ix(Operational_circuit(params), num_shots = num_shots)
    xi = measure_xi(Operational_circuit(params), num_shots = num_shots)
    iy = measure_iy(Operational_circuit(params), num_shots = num_shots)
    yi = measure_yi(Operational_circuit(params), num_shots = num_shots)
    iz = measure_iz(Operational_circuit(params), num_shots = num_shots)
    zi = measure_zi(Operational_circuit(params), num_shots = num_shots)
    xy = measure_xy(Operational_circuit(params), num_shots = num_shots)
    yx = measure_yx(Operational_circuit(params), num_shots = num_shots)
    xz = measure_xz(Operational_circuit(params), num_shots = num_shots)
    zx = measure_zx(Operational_circuit(params), num_shots = num_shots)
    yz = measure_yz(Operational_circuit(params), num_shots = num_shots)
    zy = measure_zy(Operational_circuit(params), num_shots = num_shots)
    
    coeff = coef(H)
    
    ev = (coeff[0]*1 + coeff[1]*zz + coeff[2]*xx+coeff[3]*yy+coeff[4]*ix+coeff[5]*xi+coeff[6]*iy+coeff[7]*yi+coeff[8]*iz+
         coeff[9]*zi+coeff[10]*xy+coeff[11]*yx+coeff[12]*xz+coeff[13]*zx+coeff[14]*yz+coeff[15]*zy)
    
    return ev

In [1381]:
params = [0,0,0,0,0,0,0,0,0]

In [1382]:
ev = eigen_value(params)     #Try energy with params = [0,0,0,0,0,0]
print("The Eigen_Value of the trial state is", str(ev))

The Eigen_Value of the trial state is 7.7028


# Finding Lowest Eignevalue

In [1391]:
from qiskit.aqua.components.optimizers import COBYLA,SLSQP,SPSA

optimizer = COBYLA(500,0.0001)

In [1396]:
ret = optimizer.optimize(num_vars=9, objective_function=eigen_value, initial_point=params)

In [1397]:
eigen_value(ret[0])

-0.7898299999999991

In [1398]:
ret[1]

-0.8030750000000011

In [1395]:
ret

(array([-0.3730913 ,  0.02831593, -1.22687865, -0.7829731 , -0.67909078,
        -0.36289414,  2.25296634,  0.39263271, -0.24677961]),
 1.0331100000000006,
 107)

# Eigen State

In [1388]:
backend_state = Aer.get_backend('statevector_simulator')
result = execute(Operational_circuit(ret[0]),backend_state).result()
out_state = result.get_statevector()
out_state

array([ 0.62094323+0.17464273j, -0.31620105-0.16439903j,
       -0.53948589-0.3173202j ,  0.22629065+0.1182146j ])

# Initial State

In [1389]:
expm(1j*M)

array([[ 0.20668776+0.18431628j,  0.10986512+0.47478549j,
        -0.45106743+0.57681665j, -0.23245343+0.30919607j],
       [ 0.10986512+0.47478549j,  0.48077738-0.17170243j,
         0.22467159-0.18721584j, -0.6415596 -0.06894908j],
       [-0.45106743+0.57681665j,  0.22467159-0.18721584j,
        -0.21053493-0.05681853j,  0.47631644+0.3222778j ],
       [-0.23245343+0.30919607j, -0.6415596 -0.06894908j,
         0.47631644+0.3222778j , -0.22519064+0.22925826j]])

In [1390]:
had = [0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j]
np.dot(expm(1j*M),had)

array([-0.18348399+0.77255724j,  0.08687724+0.02345906j,
        0.01969284+0.32753004j, -0.31144361+0.39589152j])